In [ ]:
import sys
sys.path.append("./notebooks")
sys.path.append("./src")

import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from statsmodels.tsa.stattools import adfuller

In [ ]:
data = np.load("../DATA/dataMonthly.npy")

### band indices:
| # | Index | Meaning (High Values ~ +1) | Meaning (Low / Negative Values ~ 0 → -1) |
|---|--------|-----------------------------|-------------------------------------------|
| 0 | **SAVI** (Soil-Adjusted Vegetation Index) | Dense, healthy vegetation; strong vigor | Bare soil, sparse or no vegetation |
| 1 | **EVI** (Enhanced Vegetation Index) | Dense, healthy canopy; reduced soil/atmosphere noise | Sparse or stressed vegetation, barren land |
| 2 | **NDRE705** (Red-Edge 705) | High chlorophyll; early healthy growth | Early stress; chlorophyll reduction begins |
| 3 | **NDRE740** (Red-Edge 740) | Balanced vegetation health | Moderate stress; photosynthetic decline |
| 4 | **NDRE783** (Red-Edge 783) | Very healthy vegetation; high biomass | Heavy stress or senescence |
| 5 | **NDVI** (Normalized Difference Vegetation Index) | Lush, green vegetation; high photosynthesis | Bare soil (~0), water or dead vegetation (<0) |
| 6 | **NDWI (Gao)** | High leaf water content; well-watered canopy | Water stress; dry or senescent leaves |
| 7 | **NDWI (McFeeters)** | Open water bodies (lakes, rivers) | Land or built-up areas; vegetation |
| 8 | **NBR** (Normalized Burn Ratio) | Healthy, unburned vegetation | Burned/disturbed areas; ash or bare soil |


In [ ]:
indices ={
    0: "SAVI",
    1: "EVI",
    2: "NDRE705",
    3: "NDRE740",
    4: "NDRE783",
    5: "NDVI",
    6: "NDWI GAO",
    7: "NDWI McFeeters",
    8: "NBR"
}

In [ ]:
import copy

def fill_zeros_1d(y):
    x = np.arange(len(y))
    zero_mask = y == 0
    if np.any(zero_mask) and np.any(~zero_mask):
        y = y.copy()
        y[zero_mask] = np.interp(x[zero_mask], x[~zero_mask], y[~zero_mask])
    return y

data_m = np.apply_along_axis(fill_zeros_1d, axis=0, arr=copy.deepcopy(data))

## Band analysis for single Pixel

In [ ]:
y = 250
x = 500
example_pixel_m = data_m[:, :, y, x]
example_pixel = data[:, :, y, x]

In [ ]:
x = np.arange(0, 69)
y = pd.Series(example_pixel_m[:, 6])
yd = y.diff(12)
y_nm = pd.Series(example_pixel[:, 6])

fig = go.Figure()
fig.add_trace(go.Scatter(x=x, y=y_nm, name="Raw band")),
fig.add_trace(go.Scatter(x=x, y=y, name="Corrected Band"))
fig.add_trace(go.Scatter(x=x, y=yd, name="Differentiated Corrected Band"))
fig.update_layout(title_text="Visualization of band correction and differentiation")
fig.show()

This visualization shows the different time series processing steps on the example of a randomly selected pixel and the Enhanced Vegetation Index (EVI).  
Cloud-related gaps are being filled using an interpolation.  
The differentiation with a lag of 12 is used to deseasonalize the time series and make changes in the index more apparent.  

Further evaluation is needed to determine for which indices this workflow is suitable, as not every index exhibits seasonality. In such cases, a simple differentiation may suffice.

## Band Analysis over complete AOI

The plots contain the p-value of the Augmented-Dickey-Fuller-Test for stationarity for the cloud-corrected series

In [ ]:
result = data_m.mean(axis=(2, 3))
for i in range(9):
    x = np.arange(0, 69)
    y = pd.Series(result[:, i])
    y_d = y.diff(12)
    
    test_result = adfuller(y)
    
    
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=x, y=y, name="Corrected"))
    fig.add_trace(go.Scatter(x=x, y=y_d, name="Differentiated"))
    fig.update_layout(title_text=f"{indices[i]} p-value: {test_result[1]}",
                      width=800,
                      height=300)
    fig.show()

Those plots indicate, that most indices don't show seasonality and already are stationary and thus the differenciation wouldn't have a significant effect.  
Plots which should be de-seasonalized:
* SAVI
* EVI

In [ ]:
# p-values of differentiation

evi = pd.Series(result[:, 0]).diff(12).dropna()
savi = pd.Series(result[:, 1]).diff(12).dropna()

evi_result = adfuller(evi)
savi_result = adfuller(savi)

print(f"p-value of EVI: {evi_result[1]}")
print(f"p-value of SAVI: {savi_result[1]}")

Those p-values now suggest that differenciating with lag 12 is a reasonable step to ensure the series are stationary for further feature extraction

## AOI Band Visualization

In [ ]:
vegetation_index = data[67, 6, :, :]

In [ ]:
fig, axes = plt.subplots(3, 3, figsize=(10, 9))

for i, ax in enumerate(axes.flat):
    im = ax.imshow(data_m[68, i, :, :]*5 if i in [3, 4] else data_m[68, i, :, :], cmap='YlGn', vmin=-1, vmax=1)
    ax.set_title(indices[i])
plt.tight_layout()

cbar = fig.colorbar(im, ax=axes.ravel().tolist(), shrink=0.4, label='Vegetation Index')
fig.set_size_inches(18, 10)

These plots show that McFeeters and NDRE783 are very homogenous and have a very low variance, which was also indicated in the time series plots above. This is why they won't be evaluated further

In [ ]:
correlation_matrix = np.corrcoef([
    data_m[:, :, 0].flatten(),
    data_m[:, :, 1].flatten(),
    data_m[:, :, 2].flatten(),
    data_m[:, :, 3].flatten(),
    data_m[:, :, 5].flatten(),
    data_m[:, :, 6].flatten(),
    data_m[:, :, 8].flatten(),
])

plt.figure(figsize=(5, 4))
plt.imshow(correlation_matrix, cmap='coolwarm', vmax=1)
plt.colorbar(label='Correlation')

for i in range(correlation_matrix.shape[0]):
    for j in range(correlation_matrix.shape[1]):
        plt.text(j, i, f'{correlation_matrix[i, j]:.2f}',
                ha='center', va='center', color='black')

plt.title('Correlation Matrix Heatmap')
plt.tight_layout()
plt.show()

# Possible Features

The feature selection is mostly based on visual evaluation whether there are patterns in the data that seem to identify the areas where it is known to have standing (dead vegetation)  
Sources: [https://www.deadtrees.earth] [https://map3d.remote-sensing-solutions.de/waldmonitor-deutschland/#].

For the clustering goal we need features that assess the current situation as well as features that capture trends.

### Mean over the last year

In [ ]:
fig, axes = plt.subplots(3, 3, figsize=(10, 9))

for i, ax in enumerate(axes.flat):
    im = ax.imshow(data_m[-12:].mean(axis=(0))[i, :, :]*5 if i in [3, 4] else data_m[-12:].mean(axis=(0))[i, :, :], cmap='YlGn', vmin=-1, vmax=1)
    ax.set_title(indices[i])
plt.tight_layout()

cbar = fig.colorbar(im, ax=axes.ravel().tolist(), shrink=0.4, label='Vegetation Index')
fig.set_size_inches(18, 10)


Here NDRE740, NDWI(Gao), NDVI and NBR seem to have an interesting structure showing the mean over the last year. NDWI and NDVI will be evaluated further as features, as they are kind of the industry standard for vegetation analysis.

---
### Difference between mean of 2020 and 2025

In [ ]:
fig, axes = plt.subplots(3, 3, figsize=(10, 9))

for i, ax in enumerate(axes.flat):
    im = ax.imshow((data_m[-12:].mean(axis=(0))-data_m[:12].mean(axis=(0)))[i, :, :]*5 if i in [3, 4] else (data_m[-12:].mean(axis=(0))-data_m[:12].mean(axis=(0)))[i, :, :], cmap='YlGn', vmin=-1, vmax=1)
    ax.set_title(indices[i])
plt.tight_layout()

cbar = fig.colorbar(im, ax=axes.ravel().tolist(), shrink=0.4, label='Vegetation Index')
fig.set_size_inches(18, 10)

This shows quite well were the forest degraded but also the spots were it got healthier. NDWI is chosen for further evaluation, because it shows the spots that got healthier quite well next to the degraded areas that are captured by nearly all indices

---
### Standard-Deviation for all features

In [ ]:
from sklearn.preprocessing import RobustScaler


fig, axes = plt.subplots(3, 3, figsize=(10, 9))

for i, ax in enumerate(axes.flat):
    index_data = np.std(data_m[:], axis=0)[i, :, :]
    rs = RobustScaler()
    data_scaled = rs.fit_transform(index_data)
    im = ax.imshow(data_scaled, cmap='YlGn', vmin=np.quantile(data_scaled, 0.01), vmax=np.quantile(data_scaled, 0.99))
    ax.set_title(indices[i])
plt.tight_layout()

cbar = fig.colorbar(im, ax=axes.ravel().tolist(), shrink=0.4, label='Vegetation Index')
fig.set_size_inches(18, 10)


The standard deviations for the non stationary indices show pretty interesting structure, where SAVI seems to be more centered than EVI which is dominated by a small section of the AOI.

The stationary indices also show quite seperating structures, that don't seem to be linked to only the areas where standing dead trees are located. As all of them seem to be quite heavily correlated, only SAVI will be chosen as a feature.

---
## Spatial Features

In [ ]:
from scipy.ndimage import gaussian_filter, generic_filter, sobel

data_spatial = data_m[-12:].mean(axis=(0))

A window size of 5 results in windows of 100x100 meters

### Local coefficient of variation

In [ ]:
def cv_func(arr):
            mean = np.mean(arr)
            std = np.std(arr)
            return std / mean if mean != 0 else 0

fig, axes = plt.subplots(3, 3, figsize=(10, 9))

for i, ax in enumerate(axes.flat):
    index_data = generic_filter(
                    data_spatial[i], cv_func, size=5, mode="constant", cval=0
                )
    rs = RobustScaler()
    data_scaled = rs.fit_transform(index_data)
    im = ax.imshow(data_scaled, cmap='YlGn', vmin=np.quantile(data_scaled, 0.01), vmax=np.quantile(data_scaled, 0.99))
    ax.set_title(indices[i])
plt.tight_layout()

cbar = fig.colorbar(im, ax=axes.ravel().tolist(), shrink=0.4, label='Vegetation Index')
fig.set_size_inches(18, 10)

### Local Heterogenity

#### STD

In [ ]:
fig, axes = plt.subplots(3, 3, figsize=(10, 9))

for i, ax in enumerate(axes.flat):
    index_data = generic_filter(
                    data_spatial[i], np.std, size=5, mode="constant", cval=0
                )
    rs = RobustScaler()
    data_scaled = rs.fit_transform(index_data)
    im = ax.imshow(data_scaled, cmap='YlGn', vmin=np.quantile(data_scaled, 0.01), vmax=np.quantile(data_scaled, 0.99))
    ax.set_title(indices[i])
plt.tight_layout()

cbar = fig.colorbar(im, ax=axes.ravel().tolist(), shrink=0.4, label='Vegetation Index')
fig.set_size_inches(18, 10)

##### Spatial STD over the difference between 2020 and 2025

In [ ]:
fig, axes = plt.subplots(3, 3, figsize=(10, 9))

for i, ax in enumerate(axes.flat):
    index_data = generic_filter(
                    (data_m[-12:].mean(axis=(0))-data_m[:12].mean(axis=(0)))[i], np.std, size=5, mode="constant", cval=0
                )
    rs = RobustScaler()
    data_scaled = rs.fit_transform(index_data)
    im = ax.imshow(data_scaled, cmap='YlGn', vmin=np.quantile(data_scaled, 0.01), vmax=np.quantile(data_scaled, 0.99))
    ax.set_title(indices[i])
plt.tight_layout()

cbar = fig.colorbar(im, ax=axes.ravel().tolist(), shrink=0.4, label='Vegetation Index')
fig.set_size_inches(18, 10)

#### Range

In [ ]:
def range_func(arr):
                return np.ptp(arr) if len(arr) > 0 else 0

fig, axes = plt.subplots(3, 3, figsize=(10, 9))

for i, ax in enumerate(axes.flat):
    index_data = generic_filter(
                    data_spatial[i], range_func, size=5, mode="constant", cval=0
                )
    rs = RobustScaler()
    data_scaled = rs.fit_transform(index_data)
    im = ax.imshow(data_scaled, cmap='YlGn', vmin=np.quantile(data_scaled, 0.01), vmax=np.quantile(data_scaled, 0.99))
    ax.set_title(indices[i])
plt.tight_layout()

cbar = fig.colorbar(im, ax=axes.ravel().tolist(), shrink=0.4, label='Vegetation Index')
fig.set_size_inches(18, 10)

### Edge-Strength

In [ ]:
def get_edge(data):
    sigma=1.0
    if sigma > 0:
        grid_2d = gaussian_filter(data, sigma=sigma)

    # Sobel operators for x and y gradients
    grad_x = sobel(grid_2d, axis=1)  # Horizontal edges
    grad_y = sobel(grid_2d, axis=0)  # Vertical edges

    # Gradient magnitude (edge strength)
    gradient_magnitude = np.sqrt(grad_x**2 + grad_y**2)
    return gradient_magnitude

fig, axes = plt.subplots(3, 3, figsize=(10, 9))

for i, ax in enumerate(axes.flat):
    index_data = get_edge(data_spatial[i])
    rs = RobustScaler()
    data_scaled = rs.fit_transform(index_data)
    im = ax.imshow(data_scaled, cmap='YlGn', vmin=np.quantile(data_scaled, 0.01), vmax=np.quantile(data_scaled, 0.99))
    ax.set_title(indices[i])
plt.tight_layout()

cbar = fig.colorbar(im, ax=axes.ravel().tolist(), shrink=0.4, label='Vegetation Index')
fig.set_size_inches(18, 10)

Out of all the spatial features, the spatial STD over the difference between 2025 and 2020 looks the most promising, as it has really high values around the edges of the degraded areas, which could identify high risk areas. NDVI, NDWI and NBR show here roughly the same, but NDVI has a bit more structure, which is why this will be evaluated further